# Create an interface between two materials with minimal strain

Use Zur and McGill superlattices matching [algorithm](https://doi.org/10.1063/1.3330840) to create interfaces between two materials using the Pymatgen [implementation](https://pymatgen.org/pymatgen.analysis.interfaces.html#pymatgen.analysis.interfaces.zsl).

<h2 style="color:green">Usage</h2>

0. Make sure to select Input Materials
1. Execute "Run first: ..." cell below to load Input Materials into the current kernel
2. Set Input Parameters (e.g. `MILLER_INDICES`, `THICKNESS`, `MAX_AREA`) below or use the default values
3. Click "Run" > "Run All" to run all cells
4. Wait for the run to complete (depending on the area, it can take 1-2 min or more). Scroll down to view cell results.
5. Review the strain plot and modify its parameters as needed

## Methodology

The following happens in the script below:

1. Create slabs for each input material. The materials data is passed in from and back to the web application according to this description (TBA).
   We assume that two input materials are either in bulk form (e.g. Ni crystal) or layered (e.g. graphene). 
   
   We construct the interface along the Z-axis. The material corresponding to the bottom of the interface is referred to as the "**substrate**", and the top - as the "**layer**". 

2. Perform strain matching on the slabs to extract the supercell dimensions. The algorithm has a set of parameters, such as the maximum area considered, that can be configured by editing the cells below.

3. When the strain matching is finished, the interface with the lowest strain (and the smallest number of atoms) is selected. We create the corresponding supercells and place them at a specified distance from each other (note no shift is performed currently).


<h2 style="color:red">Run first: load input materials in current kernel</h2>


In [1]:
# %pip install mat3ra-standata
from mat3ra.standata.materials import materials_data

materials_jsons = list(materials_data["filesMapByName"].values())


In [2]:
import re

# Search for materials with regex
pattern = r"(Si|WS2)"
materials_in = [item for item in materials_jsons if re.search(pattern, item['name'])]
materials_in[0]


{'basis': {'elements': [{'id': 0, 'value': 'Si'}, {'id': 1, 'value': 'Si'}],
  'coordinates': [{'id': 0, 'value': [0, 0, 0]},
   {'id': 1, 'value': [0.25, 0.25, 0.25]}]},
 'lattice': {'a': 3.867,
  'b': 3.867,
  'c': 3.867,
  'alpha': 60,
  'beta': 60,
  'gamma': 60,
  'type': 'TRI',
  'units': {'length': 'angstrom', 'angle': 'degree'}},
 'name': 'Si (mp-149)',
 'isNonPeriodic': False,
 'external': {'id': 'mp-149',
  'source': 'materials project',
  'doi': '10.17188/1190959',
  'url': 'https://next-gen.materialsproject.org/materials/mp-149/',
  'origin': True}}

## 1. Set Input Parameters

### 1.1. Select Substrate and Layer from Input Materials

In [3]:
SUBSTRATE_PARAMETERS = {
    "MATERIAL_INDEX": 0,  # the index of the material in the materials_in list
    "MILLER_INDICES": (1, 1, 1),  # the miller indices of the interfacial plane
    "THICKNESS": 3,  # in layers
}

LAYER_PARAMETERS = {
    "MATERIAL_INDEX": 1, # the index of the material in the materials_in list
    "MILLER_INDICES": (0, 0, 1),  # the miller indices of the interfacial plane
    "THICKNESS": 1,  # in layers
}

### 1.2. Set Interface Parameters

The distance between layer and substrate and maximum area to consider when matching.


In [4]:
INTERFACE_PARAMETERS = {
    "DISTANCE_Z": 3.0, # in Angstroms
    "MAX_AREA": 200, # in Angstroms^2
}

### 1.3. Set Algorithm Parameters

In [5]:
ZSL_PARAMETERS = {
    "MAX_AREA": INTERFACE_PARAMETERS["MAX_AREA"],  # The area to consider in Angstrom^2
    "MAX_AREA_TOL": 0.09,  # The area within this tolerance is considered equal
    "MAX_LENGTH_TOL": 0.03,  # supercell lattice vectors lengths within this tolerance are considered equal
    "MAX_ANGLE_TOL": 0.01,  # supercell lattice angles within this tolerance are considered equal
    "STRAIN_TOL": 10e-6,  # strains within this tolerance are considered equal
}

## 2. Install Packages

In [6]:
# !python --version
# !pip list
#!pip install pymatgen==2024.2.8 ase==3.22.1 nbformat==5.9.2 ipykernel==6.29.2 matgl==0.9.2 nglview==3.1.1

## 3. Create interfaces

### 3.1. Extract Interfaces and Terminations

Extract all possible layer/substrate supercell combinations within the maximum area including different terminations.

In [7]:
from src.pymatgen_coherent_interface_builder import CoherentInterfaceBuilder, ZSLGenerator
from src.utils import to_pymatgen

if "materials_in" in globals():
    pymatgen_materials = [to_pymatgen(item) for item in materials_in]
for material in pymatgen_materials:
    print(material, "\n")


def create_interfaces(settings):
    print("Creating interfaces...")
    zsl = ZSLGenerator(
        max_area_ratio_tol=settings["ZSL_PARAMETERS"]["MAX_AREA_TOL"],
        max_area=settings["ZSL_PARAMETERS"]["MAX_AREA"],
        max_length_tol=settings["ZSL_PARAMETERS"]["MAX_LENGTH_TOL"],
        max_angle_tol=settings["ZSL_PARAMETERS"]["MAX_ANGLE_TOL"],
    )

    cib = CoherentInterfaceBuilder(
        substrate_structure=pymatgen_materials[settings["SUBSTRATE_PARAMETERS"]["MATERIAL_INDEX"]],
        film_structure=pymatgen_materials[settings["LAYER_PARAMETERS"]["MATERIAL_INDEX"]],
        substrate_miller=settings["SUBSTRATE_PARAMETERS"]["MILLER_INDICES"],
        film_miller=settings["LAYER_PARAMETERS"]["MILLER_INDICES"],
        zslgen=zsl,
        strain_tol=settings["ZSL_PARAMETERS"]["STRAIN_TOL"],
    )

    # Find terminations
    cib._find_terminations()
    terminations = cib.terminations

    # Create interfaces for each termination
    interfaces = {}
    for termination in terminations:
        interfaces[termination] = []
        for interface in cib.get_interfaces(
            termination,
            gap=settings["INTERFACE_PARAMETERS"]["DISTANCE_Z"],
            film_thickness=settings["LAYER_PARAMETERS"]["THICKNESS"],
            substrate_thickness=settings["SUBSTRATE_PARAMETERS"]["THICKNESS"],
            in_layers=True,
        ):
            # Wrap atoms to unit cell
            interface["interface"].make_supercell((1,1,1), to_unit_cell=True)
            interfaces[termination].append(interface)
    return interfaces, terminations


interfaces, terminations = create_interfaces(
    settings={
        "SUBSTRATE_PARAMETERS": SUBSTRATE_PARAMETERS,
        "LAYER_PARAMETERS": LAYER_PARAMETERS,
        "ZSL_PARAMETERS": ZSL_PARAMETERS,
        "INTERFACE_PARAMETERS": INTERFACE_PARAMETERS,
    }
)

Full Formula (Si2)
Reduced Formula: Si
abc   :   3.867000   3.867000   3.867000
angles:  60.000000  60.000000  60.000000
pbc   :       True       True       True
Sites (2)
  #  SP       a     b     c
---  ----  ----  ----  ----
  0  Si    0     0     0
  1  Si    0.25  0.25  0.25 

Full Formula (W1 S2)
Reduced Formula: WS2
abc   :   3.184222   3.184222  12.978280
angles:  90.000000  90.000000 120.000000
pbc   :       True       True       True
Sites (3)
  #  SP           a         b         c
---  ----  --------  --------  --------
  0  W     0.333333  0.666667  0.120195
  1  S     0         0         0
  2  S     0         0         0.24039 

Creating interfaces...


spglib: Too many lattice symmetries was found.
        Reduce angle tolerance to 4.750000
        (line 1015, /Users/runner/work/spglib/spglib/src/symmetry.c).
spglib: Too many lattice symmetries was found.
        Reduce angle tolerance to 4.512500
        (line 1015, /Users/runner/work/spglib/spglib/src/symmetry.c).
spglib: Too many lattice symmetries was found.
        Reduce angle tolerance to 4.286875
        (line 1015, /Users/runner/work/spglib/spglib/src/symmetry.c).
spglib: Too many lattice symmetries was found.
        Reduce angle tolerance to 4.072531
        (line 1015, /Users/runner/work/spglib/spglib/src/symmetry.c).
spglib: Too many lattice symmetries was found.
        Reduce angle tolerance to 3.868905
        (line 1015, /Users/runner/work/spglib/spglib/src/symmetry.c).
spglib: Too many lattice symmetries was found.
        Reduce angle tolerance to 3.675459
        (line 1015, /Users/runner/work/spglib/spglib/src/symmetry.c).
spglib: Too many lattice symmetries was 

### 3.2. Print out the interfaces and terminations

In [8]:
print(f'Found {len(terminations)} terminations')
for termination in terminations:
    print(f"Found {len(interfaces[termination])} interfaces for", termination, "termination")

Found 1 terminations
Found 0 interfaces for ('S_P6/mmm_1', 'Si_R-3m_1') termination


## 4. Sort interfaces by strain

### 4.1. Sort all interfaces

In [9]:
# Could be "strain", "von_mises_strain", "mean_abs_strain"
strain_mode = "mean_abs_strain"

# Sort interfaces by the specified strain mode and number of sites
def sort_interfaces(interfaces, terminations):
    sorted_interfaces = {}
    for termination in terminations:
        sorted_interfaces[termination] = sorted(
            interfaces[termination], key=lambda x: (x[strain_mode], x["interface"].num_sites)
        )
    return sorted_interfaces


sorted_interfaces = sort_interfaces(interfaces, terminations)

### 4.2. Print out interfaces with lowest strain for each termination

In [10]:
for termination in terminations:
    print(f"Interface with lowest strain for termination {termination} (index 0):")
    first_interface = interfaces[termination][0]
    print("    strain:", first_interface[strain_mode] * 100, "%")
    print("    number of atoms:", first_interface["interface"].num_sites)

Interface with lowest strain for termination ('S_P6/mmm_1', 'Si_R-3m_1') (index 0):


IndexError: list index out of range

## 5. Plot the results

Plot the number of atoms vs strain. Adjust the parameters as needed.


In [ ]:
import plotly.graph_objs as go
from collections import defaultdict

PLOT_SETTINGS = {
    "HEIGHT": 600,
    "X_SCALE": "log",  # or linear
    "Y_SCALE": "log",  # or linear
}


def plot_strain_vs_atoms(sorted_interfaces, terminations, settings):
    # Create a mapping from termination to its index
    termination_to_index = {termination: i for i, termination in enumerate(terminations)}

    grouped_interfaces = defaultdict(list)
    for termination, interfaces in sorted_interfaces.items():
        for index, interface_data in enumerate(interfaces):
            strain_percentage = interface_data["mean_abs_strain"] * 100
            num_sites = interface_data["interface"].num_sites
            key = (strain_percentage, num_sites)
            grouped_interfaces[key].append((index, termination))
    
    data = []
    for (strain, num_sites), indices_and_terminations in grouped_interfaces.items():
        termination_indices = defaultdict(list)
        for index, termination in indices_and_terminations:
            termination_indices[termination].append(index)
        all_indices = [index for indices in termination_indices.values() for index in indices]
        index_range = f"{min(all_indices)}-{max(all_indices)}" if len(all_indices) > 1 else str(min(all_indices))
        
        hover_text = "<br>-----<br>".join(
             f"Termination: {termination}<br>Termination index: {termination_to_index[termination]}<br>Interfaces Index Range: {index_range}<br>Strain: {strain:.2f}%<br>Atoms: {num_sites}"
            for termination, indices in termination_indices.items()
        )
        trace = go.Scatter(
            x=[strain],
            y=[num_sites],
            text=[hover_text],
            mode="markers",
            hoverinfo="text",
            name=f"Indices: {index_range}",
        )
        data.append(trace)

    layout = go.Layout(
        xaxis=dict(title="Strain (%)", type=settings["X_SCALE"]),
        yaxis=dict(title="Number of atoms", type=settings["Y_SCALE"]),
        hovermode="closest",
        height=settings["HEIGHT"],
        legend_title_text="Interfaces Index Range",
    )
    fig = go.Figure(data=data, layout=layout)
    fig.show()



plot_strain_vs_atoms(sorted_interfaces, terminations, PLOT_SETTINGS)

for i, termination in enumerate(terminations):
    print(f"Termination {i}:", termination)

Termination 0: ('C_P6/mmm_2', 'Ni_R-3m_1')


## 6. Select the interface to relax

### 6.1. Select the interface with the desired termination and strain

The data in `sorted_interfaces` now contains an object with the following structure:

```json
{
    "('C_P6/mmm_2', 'Si_R-3m_1')": [
        { ...interface for ('C_P6/mmm_2', 'Si_R-3m_1') at index 0...},
        { ...interface for ('C_P6/mmm_2', 'Si_R-3m_1') at index 1...},
        ...
    ],
    "<termination at index 1>": [
        { ...interface for 'termination at index 1' at index 0...},
        { ...interface for 'termination at index 1' at index 1...},
        ...
    ]
}
```

Select the index for termination first, and for it - the index in the list of corresponding interfaces sorted by strain (index 0 has minimum strain).

In [ ]:
termination_index = 0
interface_index = 0

termination = terminations[termination_index]

interface = sorted_interfaces[termination][interface_index]["interface"]

## 7. Apply relaxation
### 7.1. Apply relaxation to the selected interface

In [ ]:
# Per https://github.com/materialsvirtuallab/matgl/blob/main/examples/Relaxations%20and%20Simulations%20using%20the%20M3GNet%20Universal%20Potential.ipynb

import matgl
from matgl.ext.ase import M3GNetCalculator, MolecularDynamics, Relaxer

from pymatgen.io.ase import AseAtomsAdaptor
from matgl.ext.ase import M3GNetCalculator
from matgl.apps.pes import Potential
ase_adaptor = AseAtomsAdaptor()

def get_energy(structure, pot: Potential):
    atoms = ase_adaptor.get_atoms(structure)
    calc = M3GNetCalculator(pot)
    atoms.set_calculator(calc)
    return float(atoms.get_potential_energy())

pot = matgl.load_model("M3GNet-MP-2021.2.8-PES")

relaxer = Relaxer(potential=pot, relax_cell=False)
relax_results = relaxer.relax(interface, fmax=0.01)

# extract results
relaxed_interface = relax_results["final_structure"]
relaxed_energy = relax_results["trajectory"].energies[-1]

# print out the final relaxed structure and energy
print(interface.to(fmt="poscar"))
print(relaxed_interface.to(fmt="poscar"))

print(f"The final energy is {float(relaxed_energy):.3f} eV.")

/Users/mat3ra/code/green/api-examples/.venv-3.10/lib/python3.10/site-packages/matgl/apps/pes.py:60: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).

/Users/mat3ra/code/green/api-examples/.venv-3.10/lib/python3.10/site-packages/matgl/apps/pes.py:66: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).

/Users/mat3ra/code/green/api-examples/.venv-3.10/lib/python3.10/site-packages/matgl/apps/pes.py:67: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).

/Users/mat3ra/code/green/api-examples/.venv-3.10/lib/python3.10/site-packages/matgl/layers/_basis.py:121: 

Ni3 C2
1.0
   2.4599995727812636    0.0000000000000000    0.0000000000000002
   1.2299996151168615    2.1304228614681318    0.0000000000000002
   0.0000000000000000    0.0000000000000000   27.0171630732453210
Ni C
3 2
direct
   0.3333334726277728    0.3333336565779657    0.0000000000000000 Ni
   0.0000002089416588    0.0000004848669485    0.0743446501461779 Ni
   0.6666669452555456    0.6666673131559313    0.1486893002923558 Ni
   0.0000000000000000    0.0000000000000000    0.2597298263411789 C
   0.3333330000000001    0.3333330000000000    0.2597298263411789 C

Ni3 C2
1.0
   2.4599995727812636    0.0000000000000000    0.0000000000000002
   1.2299996151168615    2.1304228614681318    0.0000000000000002
   0.0000000000000000    0.0000000000000000   27.0171630732453210
Ni C
3 2
direct
   0.3333341723412202    0.3333329825529395   -0.0173027729325095 Ni
   0.0000008748175968    0.0000011112052201    0.0577821722288098 Ni
   0.6666657896260539    0.6666680027415587    0.1327744640898099 Ni

### 7.2. View structure before and after relaxation

In [ ]:
from src.utils import poscar_to_ase

ase_structure = poscar_to_ase(relaxed_interface.to(fmt="poscar"))
ase_interface = poscar_to_ase(interface.to(fmt="poscar"))

from ase.visualize import view

# view(ase_structure, viewer="x3d")
# view(ase_interface, viewer="x3d")


### 7.3. Calculate energy energy using matgl M3GNet

In [ ]:

nickel_relaxed_interface = relaxed_interface.copy()
graphene_relaxed_interface = relaxed_interface.copy()
nickel_relaxed_interface.remove_species(["C"])
graphene_relaxed_interface.remove_species(["Ni"])

substrate_energy = get_energy(nickel_relaxed_interface, pot)
layer_energy = get_energy(graphene_relaxed_interface, pot)
delta = relaxed_energy - substrate_energy - layer_energy

print(f"Interface energy: {relaxed_energy:.4f} eV")
print(f"Substrate energy: {substrate_energy:.4f} eV")
print(f"Layer energy: {layer_energy:.4f} eV")
print(f"Substrate energy + layer energy: {substrate_energy + layer_energy:.4f} eV")
print(f"Delta: {delta:.4f} eV")


Interface energy: -34.6208 eV
Substrate energy: -16.3257 eV
Layer energy: -18.2532 eV
Substrate energy + layer energy: -34.5789 eV
Delta: -0.0419 eV
